## MLflow's Model Registry

In [ ]:
from mlflow.tracking import MlflowClient

In [ ]:
MLFLOW_TRACKING_URI = "sqlite:///../mlruns.db"

### Interacting with the MLflow tracking server

The `MlflowClient` object allows us to interact with...
- an MLflow Tracking Server that creates and manages experiments and runs.
- an MLflow Registry Server that creates and manages registered models and model versions. 

To instantiate it we need to pass a tracking URI and/or a registry URI

In [ ]:
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [ ]:
# List all experiments
# ...
experiments = client.search_experiments()
for exp in experiments:
    print(f"Experiment [{exp.experiment_id}] - {exp.name}")

In [ ]:
experiments[0]

In [ ]:
# Create a new experiment
# ...
client.create_experiment(name="my-cool-experiment!")

In [ ]:
# Soft delete new experiment
# ...
client.delete_experiment(12)

In [ ]:
# Show all experiments (with deleted ones)
# ...
client.search_experiments(view_type=1)

In [ ]:
runs = client.search_runs(experiment_ids=11)

In [ ]:
len(runs)

In [ ]:
runs[0]

In [ ]:
for run in runs:
    print(f"run id: {run.info.run_id}, train rmse: {run.data.metrics['Train RMSE']:.4f}, val rmse: {run.data.metrics['Val RMSE']}")

In [ ]:
# More detailed search
# Search for runs using filter_string and order by
# ...

from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids='7',
    filter_string="metrics.`Train RMSE` < 6",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.`Val RMSE` ASC"]
)

In [ ]:
for run in runs:
    print(f"run id: {run.info.run_id}, train rmse: {run.data.metrics['Train RMSE']:.4f}, val rmse: {run.data.metrics['Val RMSE']}")

### Interacting with the Model Registry

In this section We will use the `MlflowClient` instance to:

1. Register a model for the experiment `nyc-taxi-regressor`
2. Retrieve the latests versions of the model `nyc-taxi-regressor` and check that a new version `4` was created.
3. Transition the version `4` to "Staging" and adding annotations to it.

In [ ]:
# Create an empty registered model
# ...
REGISTERED_MODEL = "du-ai-taxi-trips"
# client.create_registered_model(REGISTERED_MODEL)

In [ ]:
# Create a registered model from a logged model
# Register best run to new model
# ...
best_run = runs[0]
run_id = best_run.info.run_id

client.create_model_version(
    name=REGISTERED_MODEL,
    source=f"runs:/{run_id}/model"
)

In [ ]:
# List all versions of registered model
# ...
model_versions = client.search_model_versions(
    filter_string=f"name = '{REGISTERED_MODEL}'"
)
model_versions

In [ ]:
# Pretty pring model versions
# ...
for version in model_versions:
    print(f"version: {version.version}, source: {version.source}")

### Updating tags and aliases from code

In [ ]:
# create "Champion" alias for version 1
# ...
# Tested model 3 in staging
client.set_registered_model_alias(REGISTERED_MODEL, "prod", 3)

In [ ]:
# get a model version by alias
# ...
client.get_model_version_by_alias(REGISTERED_MODEL, "Champion")

In [ ]:
# reassign the "Champion" alias to version 2
# ...
client.set_registered_model_alias(REGISTERED_MODEL, "Champion", 2)

In [ ]:
# get a model version by alias
# ...
client.get_model_version_by_alias(REGISTERED_MODEL, "Champion")

In [ ]:
# delete the alias
# ...
client.delete_registered_model_alias(REGISTERED_MODEL, "Champion")

In [ ]:
# get a model version by alias
# ...
client.get_model_version_by_alias(REGISTERED_MODEL, "Champion")

In [ ]:
# We can do the same with tags
# Set a tag for the registered model
# ...

# Set registered model tag
client.set_registered_model_tag(REGISTERED_MODEL, "task", "regression")

# Delete registered model tag
client.delete_registered_model_tag(REGISTERED_MODEL, "task")

In [ ]:
# Set a tag for a specific version of the registered model
# Set model version tag
client.set_model_version_tag(REGISTERED_MODEL, "1", "validation_status", "approved")

# Delete model version tag
client.delete_model_version_tag(REGISTERED_MODEL, "1", "validation_status")

### Load model

In [ ]:
import mlflow.pyfunc

In [ ]:
# Load Registered model using a specific version
# ...

model_version = 1
model_uri=f"models:/{REGISTERED_MODEL}/{model_version}"

model = mlflow.pyfunc.load_model(model_uri=model_uri)
model

In [ ]:
# Load Registered model using alias
# ...

alias = "champion"
model_uri=f"models:/{REGISTERED_MODEL}@{alias}"

champion_version = mlflow.pyfunc.load_model(model_uri=model_uri)
champion_version

### Promote Model

In [ ]:
# Set a candidate tag for a specific version
# ...
client.set_registered_model_alias(REGISTERED_MODEL, "candidate", 3)

In [ ]:
client.search_model_versions(f"name='{REGISTERED_MODEL}'")

In [ ]:
client.copy_model_version(
    src_model_uri=f"models:/{REGISTERED_MODEL}@candidate",
    dst_name="regression-model-production",
)
